In [3]:
import spacy
import pandas as pd
from afinn import Afinn
import numpy as np
from os import listdir, mkdir, path

nlp = spacy.load("en_core_web_sm")

afinn_df = pd.read_csv("data/AFINN-en-165.tsv", sep="\t", header=None, names=["word", "score"])
#afinn_df
afinn = Afinn()

vad_df = pd.read_csv("data/NRC-VAD-Lexicon.tsv", sep="\t")
#vad_df

In [4]:
afinn_scores = dict(zip(afinn_df["word"], afinn_df["score"]))
#afinn_scores["yummy"]
def afinn_score(text):
    return sum(afinn_scores.get(t.lemma_, 0) for t in nlp(text))

word_valence = dict(zip(vad_df["Word"], vad_df["Valence"]))
word_arousal = dict(zip(vad_df["Word"], vad_df["Arousal"]))
word_dminanc = dict(zip(vad_df["Word"], vad_df["Dominance"]))

def text_valence(text, agg=sum):
    return agg([word_valence.get(t.lemma_, 0) for t in nlp(text)])

def text_arousal(text, agg=sum):
    return agg([word_arousal.get(t.lemma_, 0) for t in nlp(text)])

def text_dminanc(text, agg=sum):
    return agg([word_dminanc.get(t.lemma_, 0) for t in nlp(text)])

In [ ]:
text_dir = "data/text_pp_new/"
text_files = [f for f in listdir(text_dir)]
results = []
for f in text_files:
    with open(text_dir+f, errors = "ignore") as text:
        ting = text.readlines()
        #print(len(ting))
        if len(ting) != 1:
            ting = ['no']
        #print(ting[0][0])
        score = afinn.score(ting[0])
        results.append([f[0:17], score])
        #break
print(results)

In [58]:
resultsdf = pd.DataFrame(results, columns = ['id', 'AFINN'])
resultsdf.to_csv('afinnSentimentNEW.tsv', sep='\t', index=False)

In [22]:
afinn.score('text')

0.0

In [18]:
counting = 0
score = []
text_files = ["data/text_pp/" + f for f in listdir("data/text_pp/")]
for f in text_files:
    with open(f, errors = "ignore") as txt:
        #print(row['full_text'], afinn.score(row['full_text']))
        txt = txt.readlines()[0]
        score.append([
            #txt['Datetime'],
            afinn_score(txt),
            afinn.score(txt),
            text_valence(txt),
            text_arousal(txt),
            text_dminanc(txt),
            text_valence(txt, agg=np.mean),
            text_arousal(txt, agg=np.mean),
            text_dminanc(txt, agg=np.mean),
            #txt#,
            #wordList(txt)
            ])
        #break
        counting+=1
        # if (counting % 1000 == 0): print(counting/len(tweetDf.index)*100, '%,', counting, '/', len(tweetDf.index))
        # if counting == 100: break
print(score)
#scoredf = pd.DataFrame(score, columns = ['date','AfinnSentiment','AfinnDefaultSentiment','NRCValence','NRCArousal','NRCDominance','NRCValenceMean','NRCArousalMean','NRCDominanceMean','fulltext','lemma'])